# Visualize Today's Traffic Accidents in the City of Austin
##### Miguel Novo Villar
##### December, 2022 (More to be added)

## Modules

In [32]:
#Import data science modules
import pandas as pd
import numpy as np
import matplotlib as plt

#Scraping
from sodapy import Socrata

#Time
from datetime import datetime, timedelta

#GEOSPATIAL
import geopandas as gpd
import contextily as cx
#Folium for heatmaps

import folium
from folium import plugins

## Retrieve and transform data

##### We need to get data from data.austintexas.gov and autenticate to load the entire dataset

In [33]:
MyAppToken = "MyAppTokenfrom_data.austintexas.gov"

client = Socrata("data.austintexas.gov", #website
                MyAppToken, #token from portal
                 username="mymail@gmail.com", #username
                 password="mypasswords") #password

#Retrieve from the desired repository
results = client.get("dx9v-zd7x", limit = 1000000000)

#Zips
Austin_zips = gpd.read_file("Z:\Clases\CLASE\Master Rochester\Fall_2022\InterviewPrep\ReadAustinData\Zip Code\geo_export_5b8f0432-d113-478a-b2a5-18fba0156b66.shp")[["geoid10", "geometry"]]
Austin_zips.rename(columns = {"geoid10": "ZIP"}, inplace = True)

#Convert to pandas DataFrame, timestamp and sort by date descending
results_df = pd.DataFrame.from_records(results)
results_df["published_date"] = pd.to_datetime(results_df.published_date)
results_df= results_df.sort_values(by="published_date", ascending= False)

print("All time accidents recorded: ", results_df.shape[0])

All time accidents recorded:  303343


# Transform to geopandas

Create geodataframe with accidents from today

In [34]:
today = results_df[(results_df['published_date'] > str(pd.to_datetime(datetime.utcnow() - timedelta(hours=24))))]

gdf = gpd.GeoDataFrame(today, 
                        geometry=gpd.points_from_xy(today.longitude, today.latitude),
                        crs ="EPSG:4326").drop(["location"],axis = 1)
                        # crs ="EPSG:4326").drop(["location","latitude","longitude"],axis = 1)
print("Accidents recorded today: ", today.shape[0])

Accidents recorded today:  114


## Visualize

#### Accidents today

In [35]:
accidents = gdf.drop("published_date", axis =1)

accidents.explore(column='issue_reported', legend=True, marker_kwds=dict(radius=5))

#### Accidents today - ZIP code Level

In [36]:
# Lamda function for intersecction
f = lambda x:np.sum(accidents.intersects(x))
Austin_zips["accidents_in_area"] = Austin_zips['geometry'].apply(f)

#Display intersections
Austin_zips.explore(column='accidents_in_area', legend=True, cmap  = "plasma_r")

# HotSpots and ColdSpots

In [40]:
#Center the map in Austin 30.292673,-97.685967
Austin = folium.Map(location=[30.292673,-97.685967], tiles='Stamen Toner', zoom_start=10.5)

#Add Data to the map from the intial pandas dataframe (today)
for i,row in today.iterrows():
    folium.CircleMarker((row.latitude,row.longitude), radius=3, weight=3, color='red', fill_color='red', fill_opacity=.5).add_to(Austin)

#Create heatmap
Austin.add_child(plugins.HeatMap(data=today[['latitude', 'longitude']], radius=15, blur=10, name = "HotSpots and ColdSpots of Accidents in Austin"))

#Plot Map
Austin